In [1]:
# import modules
import uproot, sys, time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import awkward as ak
from tqdm import tqdm
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as ticker

# Set up plot defaults
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 14.0,10.0  # Roughly 11 cm wde by 8 cm high
mpl.rcParams['font.size'] = 20.0 # Use 14 point font
sns.set(style="whitegrid")

font_size = {
    "xlabel": 17,
    "ylabel": 17,
    "xticks": 15,
    "yticks": 15,
    "legend": 18
}

plt.rcParams.update({
    "axes.labelsize": font_size["xlabel"],  # X and Y axis labels
    "xtick.labelsize": font_size["xticks"],  # X ticks
    "ytick.labelsize": font_size["yticks"],  # Y ticks
    "legend.fontsize": font_size["legend"]  # Legend
})

In [2]:
def getWeight(fb, sample):
    # lumi = 25767.5
    lumi = 135000
    weight = fb['mconly_weight']/fb['mc_weight_sum']*fb['xsec_ami']*fb['filter_eff_ami']*fb['kfactor_ami']*fb['pu_weight']*fb['jvt_weight']*1000*lumi
    if sample in ['ggHyyd','WH','VBF','ZH'] : 
        xsec_sig = 0.052 #if ( period == 'Run3' or 'mc23' in period ) else 0.048
        # if sample != 'ggHyyd' : xsec_sig = fb['xsec_ami']
        br = 0.01
        weight = fb['mconly_weight']/fb['mc_weight_sum']*xsec_sig*fb['pu_weight']*fb['jvt_weight']*fb['filter_eff_ami']*fb['kfactor_ami']*1000*lumi*br
    return weight

def getSampleDict():
    sample_dict = {}
    sample_dict['Zjets'] = {
        'color': 'darkgreen',   # approximates ROOT.kGreen-2
        'legend': r'Z($\nu\nu$, ll)+jets',
        'tree': 'nominal',
        'filenames': ['Zjets']
    }
    sample_dict['Zgamma'] = {
        'color': '#e6550d',      # approximates ROOT.kOrange+7
        'legend': r'Z($\nu\nu$)+$\gamma$',
        'tree': 'nominal',
        'filenames': ['Zgamma']
    }
    sample_dict['Wgamma'] = {
        'color': 'darkorange',  # approximates ROOT.kOrange+1
        'legend': r'W($l\nu$)+$\gamma$',
        'tree': 'nominal',
        'filenames': ['Wgamma']
    }
    sample_dict['Wjets'] = {
        'color': 'teal',        # approximates ROOT.kTeal+5
        'legend': r'W($l\nu$)+jets',
        'tree': 'nominal',
        'filenames': ['Wjets']
    }
    sample_dict['gammajet_direct'] = {
        'color': 'royalblue',   # approximates ROOT.kBlue+2
        'legend': r'$\gamma$+jets direct',
        'tree': 'gammajets',
        'filenames': ['gammajet_direct']
    }
    sample_dict['gammajet_frag'] = {
        'color': 'navy',        # approximates ROOT.kBlue-5
        'legend': r'$\gamma$+jets frag',
        'tree': 'gammajets',
        'filenames': ['gammajet_frag']
    }
    sample_dict['dijet'] = {
        'color': 'cyan',        # approximates ROOT.kCyan+1
        'legend': 'multijets',
        'tree': 'dijets',
        'filenames': ['dijet']
    }
    sample_dict['ggHyyd'] = {
        'color': 'red',         # approximates ROOT.kRed
        'legend': r'ggH, H$\rightarrow\gamma\gamma_{d}$',
        'tree': 'nominal',
        'filenames': ['ggHyyd']
    }
    return sample_dict
sample_dict = getSampleDict()

def getVarDict(fb, process, var_name=None):
    var_dict = {}

    # this has the same size as weight, so don't need adjustment on weighting
    if var_name is None or var_name == 'vtx_sumPt':
        var_dict['vtx_sumPt'] = {
            'var': ak.flatten(fb['vtx_sumPt']),
            'bins': np.linspace(0, 100, 20+1),  # 21 edges for 20 bins
            'title': r'vtx\_sumPt'
        }

    if var_name is None or var_name == 'n_ph':
        var_dict['n_ph'] = {
            'var': fb['n_ph'],
            'bins': np.linspace(0, 7, 7+1),
            'title': r'$N_{ph}$'
        }

    if var_name is None or var_name == 'n_ph_baseline':
        var_dict['n_ph_baseline'] = {
            'var': fb['n_ph_baseline'],
            'bins': np.linspace(0, 7, 7+1),
            'title': r'$N_{ph\_baseline}$'
        }

    if var_name is None or var_name == 'n_el_baseline':
        var_dict['n_el_baseline'] = {
            'var': fb['n_el_baseline'],
            'bins': np.linspace(0, 7, 7+1),
            'title': r'$N_{el\_baseline}$'
        }

    if var_name is None or var_name == 'n_mu_baseline':
        var_dict['n_mu_baseline'] = {
            'var': fb['n_mu_baseline'],
            'bins': np.linspace(0, 7, 7+1),
            'title': r'$N_{mu\_baseline}$'
        }

    if var_name is None or var_name == 'n_tau_baseline':
        var_dict['n_tau_baseline'] = {
            'var': fb['n_tau_baseline'],
            'bins': np.linspace(0, 7, 7+1),
            'title': r'$N_{tau\_baseline}$'
        }

    if var_name is None or var_name == 'puWeight':
        var_dict['puWeight'] = {
            'var': fb['pu_weight'],
            'bins': np.linspace(0, 2, 50+1),
            'title': r'PU weight',
            'shift': '+0'
        }

    if var_name is None or var_name == 'actualIntPerXing':
        var_dict['actualIntPerXing'] = {
            'var': fb['actualIntPerXing'],
            'bins': np.linspace(0, 100, 50+1),
            'title': r'$\langle\mu\rangle$',
            'shift': '+0'
        }

    if var_name is None or var_name == 'mt':
        var_dict['mt'] = {
            'var': np.sqrt(2 * fb['met_tst_et'] * ak.firsts(fb['ph_pt']) * 
                           (1 - np.cos(fb['met_tst_phi'] - ak.firsts(fb['ph_phi'])))) / 1000,
            'bins': np.linspace(0, 300, 15+1),
            'title': r'$m_T\ [GeV]$',
            'shift': '+0'
        }

    if var_name is None or var_name == 'metsig':
        var_dict['metsig'] = {
            'var': fb['met_tst_sig'],
            'bins': np.linspace(0, 30, 15+1),
            'title': r'$E_T^{miss}\ significance$',
            'shift': '*1'
        }

    if var_name is None or var_name == 'metsigres':
        var_dict['metsigres'] = {
            'var': fb['met_tst_et'] / fb['met_tst_sig'],
            'bins': np.linspace(0, 100000, 50+1),
            'title': r'$E_T^{miss}\ significance$',
            'shift': '*1'
        }

    if var_name is None or var_name == 'met':
        var_dict['met'] = {
            'var': fb['met_tst_et'],
            'bins': np.linspace(0, 300000, 50+1),
            'title': r'$E_T^{miss}\ [GeV]$',
            'shift': '+50000'
        }

    if var_name is None or var_name == 'met_noJVT':
        var_dict['met_noJVT'] = {
            'var': fb['met_tst_noJVT_et'],
            'bins': np.linspace(0, 300000, 50+1),
            'title': r'$E_T^{miss}\ [GeV]$'
        }

    if var_name is None or var_name == 'met_cst':
        var_dict['met_cst'] = {
            'var': fb['met_cst_et'],
            'bins': np.linspace(0, 300000, 50+1),
            'title': r'$E_T^{miss}\ CST\ [GeV]$'
        }

    if var_name is None or var_name == 'met_track':
        var_dict['met_track'] = {
            'var': fb['met_track_et'],
            'bins': np.linspace(0, 300000, 50+1),
            'title': r'$E_T^{miss}\ Track\ [GeV]$'
        }

    if var_name is None or var_name == 'dmet':
        var_dict['dmet'] = {
            'var': fb['met_tst_noJVT_et'] - fb['met_tst_et'],
            'bins': np.linspace(-100000, 100000, 20+1),
            'title': r'$E_{T,\mathrm{noJVT}}^{miss}-E_T^{miss}\ [GeV]$',
            'shift': '*1'
        }

    if var_name is None or var_name == 'ph_pt':
        var_dict['ph_pt'] = {
            'var': ak.firsts(fb['ph_pt']),
            'bins': np.linspace(0, 300000, 50+1),
            'title': r'$p_T^{\gamma}\ [GeV]$',
            'shift': '-150000'
        }

    if var_name is None or var_name == 'ph_eta':
        var_dict['ph_eta'] = {
            'var': np.abs(ak.firsts(fb['ph_eta'])),
            'bins': np.linspace(0, 4, 16+1),
            'title': r'$\eta^{\gamma}$'
        }

    if var_name is None or var_name == 'ph_phi':
        var_dict['ph_phi'] = {
            'var': ak.firsts(fb['ph_phi']),
            'bins': np.linspace(-4, 4, 50+1),
            'title': r'$\phi^{\gamma}$'
        }

    if var_name is None or var_name == "jet_central_eta":
        jet_central_eta_tmp = ak.firsts(fb['jet_central_eta'])
        var_dict['jet_central_eta'] = {
            'var': ak.fill_none(jet_central_eta_tmp, -999),
            'bins': np.linspace(-4, 4, 50+1), 
            'title': r'$\eta^{\mathrm{jets}}$'
        }

    # Jet central pt1 (first jet)
    if var_name is None or var_name == "jet_central_pt1":
        jet_central_pt1_tmp = ak.firsts(fb['jet_central_pt'])
        var_dict['jet_central_pt1'] = {
            'var': ak.fill_none(jet_central_pt1_tmp, -999),
            'bins': np.linspace(0, 300000, 50+1),
            'title': r'$p_T^{j1}\ [GeV]$'
        }

    # Jet central pt2 (second jet, if available)
    if var_name is None or var_name == "jet_central_pt2":
        jet_central_pt2_tmp = ak.mask(fb['jet_central_pt'], ak.num(fb['jet_central_pt']) >= 2)[:, 1]
        var_dict['jet_central_pt2'] = {
            'var': ak.fill_none(jet_central_pt2_tmp, -999),
            'bins': np.linspace(0, 300000, 50+1),
            'title': r'$p_T^{j2}\ [GeV]$'
        }

    # Jet central pt (all jets)
    if var_name is None or var_name == "jet_central_pt":
        weight_tmp = getWeight(fb, process)
        expanded_weights = ak.flatten(ak.broadcast_arrays(weight_tmp, fb['jet_central_pt'])[0])
        var_dict['jet_central_pt'] = {
            'var': ak.flatten(fb['jet_central_pt']),
            'weight': expanded_weights,
            'bins': np.linspace(0, 300000, 50+1),
            'title': r'$p_T^{j}\ [GeV]$'
    }

    if var_name is None or var_name == 'dphi_met_phterm':
        var_dict['dphi_met_phterm'] = {
            'var': np.arccos(np.cos(fb['met_tst_phi'] - fb['met_phterm_phi'])),
            'bins': np.linspace(0, 4, 16+1),
            'title': r'$\Delta\phi(E_T^{miss},\, E_T^{\gamma})$',
            'shift': '+0'
        }

    if var_name is None or var_name == 'dphi_met_ph':
        var_dict['dphi_met_ph'] = {
            'var': np.arccos(np.cos(fb['met_tst_phi'] - ak.firsts(fb['ph_phi']))),
            'bins': np.linspace(0, 4, 50+1),
            'title': r'$\Delta\phi(E_T^{miss},\, E_T^{\gamma})$'
        }

    if var_name is None or var_name == 'dphi_met_jetterm':
        var_dict['dphi_met_jetterm'] = {
            'var': np.where(fb['met_jetterm_et'] != 0,
                            np.arccos(np.cos(fb['met_tst_phi'] - fb['met_jetterm_phi'])),
                            -999),
            'bins': np.linspace(0, 4, 16+1),
            'title': r'$\Delta\phi(E_T^{miss},\, E_T^{jet})$'
        }

    if var_name is None or var_name == 'dphi_phterm_jetterm':
        var_dict['dphi_phterm_jetterm'] = {
            'var': np.where(fb['met_jetterm_et'] > 0,
                            np.arccos(np.cos(fb['met_phterm_phi'] - fb['met_jetterm_phi'])),
                            -999),
            'bins': np.linspace(0, 4, 50+1),
            'title': r'$\Delta\phi(E_T^{\gamma},\, E_T^{jet})$'
        }

    # Delta phi (photon vs. central jet1)
    if var_name is None or var_name == 'dphi_ph_centraljet1':
        dphi_ph_centraljet1_tmp = np.arccos(np.cos(ak.firsts(fb['ph_phi']) - ak.firsts(fb['jet_central_phi'])))
        var_dict['dphi_ph_centraljet1'] = {
            'var': ak.fill_none(dphi_ph_centraljet1_tmp, -999),
            'bins': np.linspace(0, 4, 50+1),
            'title': r'$\Delta\phi(\gamma,\, j1)$'
        }

    # # Delta phi (central jet1 vs. jet2) (repeated with dphi_jj)
    # if var_name is None or var_name == 'dphi_central_jet1_jet2':
    #     phi1_tmp = ak.firsts(fb['jet_central_phi'])
    #     phi2_tmp = ak.mask(fb['jet_central_phi'], ak.num(fb['jet_central_phi']) >= 2)[:, 1]
    #     dphi_central_tmp = np.arccos(np.cos(phi1_tmp - phi2_tmp))
    #     var_dict['dphi_central_jet1_jet2'] = {
    #         'var': ak.fill_none(dphi_central_tmp, -999),
    #         'bins': np.linspace(0, 4, 50+1),
    #         'title': r'$\Delta\phi(j1,\, j2)$'
    #     }

    # Met plus photon pt
    if var_name is None or var_name == 'metplusph':
        var_dict['metplusph'] = {
            'var': fb['met_tst_et'] + ak.firsts(fb['ph_pt']),
            'bins': np.linspace(0, 300000, 50+1),
            'title': r'$E_T^{miss}+p_T^{\gamma}\ [GeV]$'
        }

    # # Fail JVT jet pt (all)
    if var_name is None or var_name == 'failJVT_jet_pt':
        weight_tmp = getWeight(fb, process)
        expanded_weights = ak.flatten(ak.broadcast_arrays(weight_tmp, fb['failJVT_jet_pt'])[0])
        var_dict['failJVT_jet_pt'] = {
            'var': ak.flatten(fb['failJVT_jet_pt']),
            'weight': expanded_weights,
            'bins': np.linspace(0, 300000, 50+1),
            'title': r'$p_T^{\mathrm{noJVT\ jet}}\ [GeV]$'
        }

    # # Fail JVT jet pt1 (first element)
    if var_name is None or var_name == 'failJVT_jet_pt1':
        failJVT_jet_pt_tmp = ak.firsts(fb['failJVT_jet_pt'])
        var_dict['failJVT_jet_pt1'] = {
            'var': ak.fill_none(failJVT_jet_pt_tmp, -999),
            'bins': np.linspace(20000, 60000, 40+1),
            'title': r'$p_T^{\mathrm{noJVT\ jet1}}\ [GeV]$'
        }

    if var_name is None or var_name == 'softerm':
        var_dict['softerm'] = {
            'var': fb['met_softerm_tst_et'],
            'bins': np.linspace(0, 100000, 50+1),
            'title': r'$E_T^{soft}\ [GeV]$'
        }

    if var_name is None or var_name == 'jetterm':
        var_dict['jetterm'] = {
            'var': fb['met_jetterm_et'],
            'bins': np.linspace(0, 300000, 50+1),
            'title': r'$E_T^{jet}\ [GeV]$'
        }

    if var_name is None or var_name == 'jetterm_sumet':
        var_dict['jetterm_sumet'] = {
            'var': fb['met_jetterm_sumet'],
            'bins': np.linspace(0, 300000, 50+1),
            'title': r'$E_T^{jet}\ [GeV]$'
        }

    if var_name is None or var_name == 'n_jet':
        var_dict['n_jet'] = {
            'var': fb['n_jet'],
            'bins': np.linspace(0, 10, 10+1),
            'title': r'$N_{jet}$'
        }

    if var_name is None or var_name == 'n_jet_central':
        var_dict['n_jet_central'] = {
            'var': fb['n_jet_central'],
            'bins': np.linspace(0, 10, 10+1),
            'title': r'$N_{jet}^{central}$'
        }

    if var_name is None or var_name == 'n_jet_fwd':
        var_dict['n_jet_fwd'] = {
            'var': fb['n_jet'] - fb['n_jet_central'],
            'bins': np.linspace(0, 10, 10+1),
            'title': r'$N_{jet}^{fwd}$'
        }

    # if var_name is None or var_name == 'vertex':
    #     var_dict['vertex'] = {
    #         'var': (np.abs(ak.firsts(fb['pv_truth_z']) - ak.firsts(fb['pv_z'])) == 
    #                 np.min(np.abs(ak.firsts(fb['pv_truth_z']) - fb['pv_z']))),
    #         'bins': np.linspace(0, 2, 2+1),
    #         'title': r'good PV'
    #     }

    if var_name is None or var_name == 'goodPV':
        var_dict['goodPV'] = {
            'var': (np.abs(ak.firsts(fb['pv_truth_z']) - ak.firsts(fb['pv_z'])) <= 0.5),
            'bins': np.linspace(0, 2, 2+1),
            'title': r'good PV'
        }
    # # Delta phi (met vs. central jet)
    if var_name is None or var_name == 'dphi_met_central_jet':
        dphi_met_central_jet_tmp = np.arccos(np.cos(fb['met_tst_phi'] - ak.firsts(fb['jet_central_phi'])))
        var_dict['dphi_met_central_jet'] = {
            'var': ak.fill_none(dphi_met_central_jet_tmp, -999),
            'bins': np.linspace(0, 4, 50+1),
            'title': r'$\Delta\phi(E_T^{miss},\, jet)$'
        }

    # # Counts: constant 0.5 (typically used for normalization)
    # var_dict['counts'] = {
    #     'var': 0.5,
    #     'bins': np.linspace(0, 1, 1+1),
    #     'title': ''
    # }

    # # Jet central timing1
    if var_name is None or var_name == 'jet_central_timing1':
        jet_central_timing1_tmp = ak.firsts(fb['jet_central_timing'])
        var_dict['jet_central_timing1'] = {
            'var': ak.fill_none(jet_central_timing1_tmp, -999),
            'bins': np.linspace(-40, 40, 50+1),
            'title': r'$Jet\ timing$'
        }

    # # Jet central timing (all)
    if var_name is None or var_name == 'jet_central_timing':
        weight_tmp = getWeight(fb, process)
        expanded_weights = ak.flatten(ak.broadcast_arrays(weight_tmp, fb['jet_central_timing'])[0])
        var_dict['jet_central_timing'] = {
            'var': ak.flatten(fb['jet_central_timing']),
            'weight': expanded_weights,
            'bins': np.linspace(-40, 40, 50+1),
            'title': r'$Jet\ timing$'
        }

    # # Jet central EM fraction
    if var_name is None or var_name == 'jet_central_emfrac':
        weight_tmp = getWeight(fb, process)
        expanded_weights = ak.flatten(ak.broadcast_arrays(weight_tmp, fb['jet_central_emfrac'])[0])
        var_dict['jet_central_emfrac'] = {
            'var': ak.flatten(fb['jet_central_emfrac']),
            'bins': np.linspace(-1, 2, 50+1),
            'title': r'$Jet\ EM\ fraction$'
        }

    if var_name is None or var_name == 'jet_central_emfrac':
        jet_central_emfrac1_tmp = ak.firsts(fb['jet_central_emfrac'])
        var_dict['jet_central_emfrac'] = {
            'var': ak.fill_none(jet_central_emfrac1_tmp, -999),
            'bins': np.linspace(-1, 2, 50+1),
            'title': r'$Jet\ EM\ fraction$'
        }


    # Balance: (met_tst_et+ph_pt[0]) divided by the sum over jet_central_pt.
    if var_name is None or var_name == 'balance':
        jet_sum_tmp = ak.sum(fb['jet_central_pt'], axis=-1)
        expr = (fb['met_tst_et'] + ak.firsts(fb['ph_pt'])) / ak.where(jet_sum_tmp != 0, jet_sum_tmp, 1)
        balance = ak.where(jet_sum_tmp != 0, expr, -999) 

        var_dict['balance'] = {
            'var': balance,
            'bins': np.linspace(0, 20, 100+1),
            'title': r'balance'
        }

    if var_name is None or var_name == 'balance_sumet':
        sumet_tmp = fb['met_jetterm_sumet']
        expr = (fb['met_tst_et'] + ak.firsts(fb['ph_pt'])) / ak.where(sumet_tmp != 0, sumet_tmp, 1)
        balance_sumet = ak.where(sumet_tmp != 0, expr, -999)

        var_dict['balance_sumet'] = {
            'var': balance_sumet,
            'bins': np.linspace(0, 80, 80+1),
            'title': r'balance'
        }

    if var_name is None or var_name == 'central_jets_fraction':
        var_dict['central_jets_fraction'] = {
            'var': np.where(fb['n_jet'] > 0, fb['n_jet_central']/fb['n_jet'], -1),
            'bins': np.linspace(-1, 2, 50+1),
            'title': r'Central jets fraction'
        }

    if var_name is None or var_name == 'trigger':
        var_dict['trigger'] = {
            'var': fb['trigger_HLT_g50_tight_xe40_cell_xe70_pfopufit_80mTAC_L1eEM26M'],
            'bins': np.linspace(0, 2, 2+1),
            'title': r'Pass Trigger'
        }

    # dphi_jj: Use Alt$ logic – if jet_central_phi has at least two entries, compute the difference; else -1.
    # Here we use a Python conditional (this assumes fb['jet_central_phi'] is an array with shape information).
    if var_name is None or var_name == 'dphi_jj':
        phi1_tmp = ak.firsts(fb['jet_central_phi'])
        phi2_tmp = ak.mask(fb['jet_central_phi'], ak.num(fb['jet_central_phi']) >= 2)[:, 1]
        dphi_tmp = np.arccos(np.cos(phi1_tmp - phi2_tmp))
        var_dict['dphi_jj'] = {
            'var': ak.fill_none(dphi_tmp, -999),
            'bins': np.linspace(-1, 4, 20+1),
            'title': r'$\Delta\phi(j1,\, j2)$'
        }
    
    if var_name is None or var_name == 'BDTScore':
        var_dict['BDTScore'] = {
            'var': fb['BDTScore'],
            'bins': np.arange(0, 1+0.1, 0.1),
            'title': 'BDTScore'
        }
    
    return var_dict

In [3]:
!ls /data/tmathew/ntups/mc23d

dijet_y.root		ggHyyd_y.root	VHyyd_y.root   Zgamma_y.root
gammajet_direct_y.root	qqZHyyd_y.root	Wgamma_y.root  Zjets_y.root
gammajet_frag_y.root	VBFHyyd_y.root	Wjets_y.root


In [3]:
path = "/data/tmathew/ntups/mc23d/ggHyyd_y.root"
f = uproot.open(path)['nominal']
f.keys()

['run',
 'randomRunNumber',
 'event',
 'year',
 'averageIntPerXing',
 'actualIntPerXing',
 'corAverageIntPerXing',
 'corActualIntPerXing',
 'trigger_HLT_g100_loose_L1EM22VHI',
 'trigger_HLT_g100_loose_L1eEM26M',
 'trigger_HLT_g100_loose_L1eEM28M',
 'trigger_HLT_g120_loose_L1EM22VHI',
 'trigger_HLT_g120_loose_L1eEM26M',
 'trigger_HLT_g120_loose_L1eEM28MHLT_g25_loose_L1EM20VH',
 'trigger_HLT_g140_loose_L1EM22VHI',
 'trigger_HLT_g140_loose_L1eEM26M',
 'trigger_HLT_g25_loose_L1eEM24L',
 'trigger_HLT_g30_loose_L1EM20VH',
 'trigger_HLT_g30_loose_L1eEM24L',
 'trigger_HLT_g40_loose_L1EM20VH',
 'trigger_HLT_g40_loose_L1eEM24L',
 'trigger_HLT_g50_loose_L1EM20VH',
 'trigger_HLT_g50_loose_L1eEM24L',
 'trigger_HLT_g50_tight_xe40_cell_xe50_pfopufit_80mTAC_EM22VHI',
 'trigger_HLT_g50_tight_xe40_cell_xe50_pfopufit_80mTAC_L1eEM26M',
 'trigger_HLT_g50_tight_xe40_cell_xe60_pfopufit_80mTAC_EM22VHI',
 'trigger_HLT_g50_tight_xe40_cell_xe60_pfopufit_80mTAC_L1eEM26M',
 'trigger_HLT_g50_tight_xe40_cell_xe70_pf

In [ ]:
variables = [
    "actualIntPerXing", "failJVT_jet_pt", "jet_central_emfrac", "jet_central_eta",
    "jet_central_phi", "jet_central_pt", "jet_central_timing", "met_cst_et",
    "met_jetterm_et", "met_jetterm_phi", "met_jetterm_sumet", "met_phterm_phi",
    "met_softerm_tst_et", "met_tst_et", "met_tst_noJVT_et", "met_tst_phi",
    "met_tst_sig", "met_track_et", "n_ph", "n_ph_baseline", "n_el_baseline",
    "n_mu_baseline", "n_jet", "n_jet_central", "n_tau_baseline", "ph_eta",
    "ph_phi", "ph_pt", "pu_weight", "pv_truth_z", "pv_z",
    "trigger_HLT_g50_tight_xe40_cell_xe70_pfopufit_80mTAC_L1eEM26M", "vtx_sumPt",
    "mconly_weight", "mc_weight_sum", "xsec_ami", "filter_eff_ami", "kfactor_ami",
    "pu_weight", "jvt_weight", "event"
]
ntuple_name = ['ggHyyd','Zjets','Zgamma','Wgamma','Wjets','gammajet_direct','gammajet_frag','dijet']
ntuple_name_BDT = ['ggHyyd','Zjets','Zgamma','Wgamma','Wjets','gammajet_direct','gammajet_frag','dijets']

tot = []
data = pd.DataFrame()
unweighted_bcut, weighted_bcut, unweighted_acut, weighted_acut = [], [], [], []

def test(fb):
    # checking if there are any none values
    mask = ak.is_none(fb['met_tst_et'])
    n_none = ak.sum(mask)
    print("Number of none values: ", n_none)
    # if n_none > 0:
    #     fb = fb[~mask]
    # print("Events after removing none values: ", len(fb), ak.sum(ak.is_none(fb['met_tst_et'])))

# i = 0
for i in range(len(ntuple_name)):
    cut = []
    start_time = time.time()
    path = f"/data/tmathew/ntups/mc23d/{ntuple_name[i]}_y.root" 
    path_BDT = f"/data/fpiazza/ggHyyd/Ntuples/MC23d/withVertexBDT/mc23d_{ntuple_name_BDT[i]}_y_BDT_score.root" 
    print('processing file: ', path)
    f = uproot.open(path)['nominal']
    fb = f.arrays(variables, library="ak")

    # add BDT score to fb
    f_BDT = uproot.open(path_BDT)['nominal']
    fb_BDT = f_BDT.arrays(["event", "BDTScore"], library="ak")
    tmp = fb["event"] == fb_BDT["event"]
    if np.all(tmp) == True:
        fb["BDTScore"] = fb_BDT["BDTScore"]
    else: 
        print("Something is wrong, need arranging")
    

    print("Unweighted Events before cut: ", len(fb))
    print("Weighted Events before cut: ", sum(getWeight(fb, ntuple_name[i])))
    unweighted_bcut.append(len(fb))
    weighted_bcut.append(sum(getWeight(fb, ntuple_name[i])))


    fb = fb[fb['n_ph_baseline'] == 1]
    fb = fb[fb['n_ph'] == 1]
    fb = fb[fb['n_mu_baseline'] == 0]
    fb = fb[fb['n_el_baseline'] == 0]
    fb = fb[fb['n_tau_baseline'] == 0]
    fb = fb[fb['trigger_HLT_g50_tight_xe40_cell_xe70_pfopufit_80mTAC_L1eEM26M']==1]
    fb = fb[ak.num(fb['ph_pt']) > 0] # prevent none values in Tbranch
    fb = fb[fb['met_tst_et'] >= 100000] # MET cut (basic cut)
    fb = fb[ak.firsts(fb['ph_pt']) >= 50000] # ph_pt cut (basic cut)
    fb = fb[fb['n_jet_central'] <= 4] # n_jet_central cut (basic cut)

    # goodPV on signal only
    if ntuple_name[i] == 'ggHyyd':
        fb = fb[ak.num(fb['pv_z']) > 0]
        good_pv_tmp = (np.abs(ak.firsts(fb['pv_truth_z']) - ak.firsts(fb['pv_z'])) <= 0.5)
        fb = fb[good_pv_tmp]

    mt_tmp = np.sqrt(2 * fb['met_tst_et'] * ak.firsts(fb['ph_pt']) * 
                            (1 - np.cos(fb['met_tst_phi'] - ak.firsts(fb['ph_phi'])))) / 1000
    mask1 = mt_tmp >= 80 # trigger cut
    fb = fb[mask1]
    cut.append(len(fb))

    fb = fb[fb['BDTScore'] >= 0.1] # added cut 1
    cut.append(len(fb))

    # metsig_tmp = fb['met_tst_sig'] # added cut 2 
    # mask1 = metsig_tmp >= 7
    # mask2 = metsig_tmp <= 16
    # fb = fb[mask1 * mask2]
    # cut.append(len(fb))

    # dphi_met_phterm_tmp = np.arccos(np.cos(fb['met_tst_phi'] - fb['met_phterm_phi'])) # added cut 3
    # fb = fb[dphi_met_phterm_tmp >= 1.3]
    # cut.append(len(fb))

    # dmet_tmp = fb['met_tst_noJVT_et'] - fb['met_tst_et'] # added cut 4
    # mask1 = dmet_tmp >= -20000
    # mask2 = dmet_tmp <= 50000
    # fb = fb[mask1 * mask2]
    # cut.append(len(fb))

    # dphi_met_jetterm_tmp = np.where(fb['met_jetterm_et'] != 0,   # added cut 5
    #                         np.arccos(np.cos(fb['met_tst_phi'] - fb['met_jetterm_phi'])),
    #                         -999)
    # fb = fb[dphi_met_jetterm_tmp <= 0.75]

    # ph_eta_tmp = np.abs(ak.firsts(fb['ph_eta'])) # added cut 6
    # fb = fb[ph_eta_tmp <= 1.75]
    
    # # dphi_ph_centraljet1_tmp = np.arccos(np.cos(ak.firsts(fb['ph_phi']) - ak.firsts(fb['jet_central_phi']))) # added cut 4
    # # dphi_ph_centraljet1_tmp = ak.fill_none(dphi_ph_centraljet1_tmp, -999)
    # # valid_mask = dphi_ph_centraljet1_tmp != -999 # keeping -999 values
    # # dphi_ph_centraljet1 = ak.mask(dphi_ph_centraljet1_tmp, (dphi_ph_centraljet1_tmp >= 1.5) | ~valid_mask)
    # # fb = fb[~ak.is_none(dphi_ph_centraljet1)]
    # # cut.append(len(fb))

    # phi1_tmp = ak.firsts(fb['jet_central_phi']) # added cut 7
    # phi2_tmp = ak.mask(fb['jet_central_phi'], ak.num(fb['jet_central_phi']) >= 2)[:, 1] 
    # dphi_tmp = np.arccos(np.cos(phi1_tmp - phi2_tmp))
    # dphi_jj_tmp = ak.fill_none(dphi_tmp, -1)
    # fb = fb[dphi_jj_tmp <= 2.5]
    # cut.append(len(fb))


    print("Unweighted Events after cut: ", len(fb))
    print("Weighted Events after cut: ", sum(getWeight(fb, ntuple_name[i])))
    cut.append(len(fb))

    unweighted_acut.append(cut)
    # unweighted_acut.append(len(fb))
    weighted_acut.append(sum(getWeight(fb, ntuple_name[i])))
    test(fb) # check for none value

    print(f"Reading Time for {ntuple_name[i]}: {(time.time()-start_time)} seconds\n")



    tot.append(fb)

    fb = 0
    fb_BDT = 0
    tmp = 0


processing file:  /data/tmathew/ntups/mc23d/ggHyyd_y.root
Unweighted Events before cut:  225374
Weighted Events before cut:  22918.521
Unweighted Events after cut:  4328
Weighted Events after cut:  438.0519
Number of none values:  0
Reading Time for ggHyyd: 1.8977868556976318 seconds

processing file:  /data/tmathew/ntups/mc23d/Zjets_y.root
Unweighted Events before cut:  11544081
Weighted Events before cut:  1739103.6
Unweighted Events after cut:  14837
Weighted Events after cut:  869.52246
Number of none values:  0
Reading Time for Zjets: 108.18025827407837 seconds

processing file:  /data/tmathew/ntups/mc23d/Zgamma_y.root
Unweighted Events before cut:  4341141
Weighted Events before cut:  310120.2
Unweighted Events after cut:  709387
Weighted Events after cut:  17007.615
Number of none values:  0
Reading Time for Zgamma: 42.32181429862976 seconds

processing file:  /data/tmathew/ntups/mc23d/Wgamma_y.root
Unweighted Events before cut:  1772654
Weighted Events before cut:  567686.56
Un

In [40]:
def sel(tot):
    tot2 = []
    for i in range(len(tot)):
        fb2 = tot[i]
        
        metsig_tmp = fb2['met_tst_sig'] # added cut 2 
        mask1 = metsig_tmp >= 7
        fb2 = fb2[mask1]

        dphi_met_phterm_tmp = np.arccos(np.cos(fb2['met_tst_phi'] - fb2['met_phterm_phi'])) # added cut 3
        fb2 = fb2[dphi_met_phterm_tmp >= 1.35]

        dmet_tmp = fb2['met_tst_noJVT_et'] - fb2['met_tst_et'] # added cut 4
        mask1 = dmet_tmp >= -20000
        mask2 = dmet_tmp <= 50000
        fb2 = fb2[mask1 * mask2]

        dphi_met_jetterm_tmp = np.where(fb2['met_jetterm_et'] != 0,   # added cut 5
                                np.arccos(np.cos(fb2['met_tst_phi'] - fb2['met_jetterm_phi'])),
                                -999)
        fb2 = fb2[dphi_met_jetterm_tmp <= 0.70]

        ph_eta_tmp = np.abs(ak.firsts(fb2['ph_eta'])) # added cut 6
        fb2 = fb2[ph_eta_tmp <= 1.75]


        phi1_tmp = ak.firsts(fb2['jet_central_phi']) # added cut 7
        phi2_tmp = ak.mask(fb2['jet_central_phi'], ak.num(fb2['jet_central_phi']) >= 2)[:, 1] 
        dphi_tmp = np.arccos(np.cos(phi1_tmp - phi2_tmp))
        dphi_jj_tmp = ak.fill_none(dphi_tmp, -999)
        fb2 = fb2[dphi_jj_tmp <= 2.5]

        jet_sum_tmp = ak.sum(fb2['jet_central_pt'], axis=-1)
        expr = (fb2['met_tst_et'] + ak.firsts(fb2['ph_pt'])) / ak.where(jet_sum_tmp != 0, jet_sum_tmp, 1)
        balance_tmp = ak.where(jet_sum_tmp != 0, expr, 999) 
        fb2 = fb2[balance_tmp >= 0.65]

        mt_tmp = np.sqrt(2 * fb2['met_tst_et'] * ak.firsts(fb2['ph_pt']) * 
                            (1 - np.cos(fb2['met_tst_phi'] - ak.firsts(fb2['ph_phi'])))) / 1000
        mask1 = mt_tmp >= 95
        fb2 = fb2[mask1]

        
        tot2.append(fb2)
    return tot2

tot2 = sel(tot)

In [44]:
Vars = [
    'metsig',
    'metsigres',
    'met',
    'met_noJVT',
    'dmet',
    'ph_pt',
    'ph_eta',
    'ph_phi',
    'jet_central_eta',
    'jet_central_pt1',
    'jet_central_pt2',
    'dphi_met_phterm',
    'dphi_met_ph',
    'dphi_met_jetterm',
    'dphi_phterm_jetterm',
    'dphi_ph_centraljet1',
    'metplusph',
    'failJVT_jet_pt1',
    'softerm',
    'jetterm',
    'jetterm_sumet',
    'n_jet_central',
    'dphi_met_central_jet',
    'balance',
    'dphi_jj',
    'BDTScore',
    'mt'
]

ntuple_name = ['ggHyyd','Zjets','Zgamma','Wgamma','Wjets','gammajet_direct','gammajet_frag','dijet']
data_list = []

for j in range(len(ntuple_name) - 1):
    process = ntuple_name[j]
    fb = tot2[j] 
    
    data_dict = {}
    
    for var in Vars:
        var_config = getVarDict(fb, process, var_name=var)
        data_dict[var] = var_config[var]['var']
    
    weights = getWeight(fb, process)
    data_dict['weights'] = weights
    
    n_events = len(weights)
    data_dict['process'] = [process] * n_events
    label = 1 if process == 'ggHyyd' else 0
    data_dict['label'] = [label] * n_events
    
    df_temp = pd.DataFrame(data_dict)
    data_list.append(df_temp)

df_all = pd.concat(data_list, ignore_index=True)
df_all.head()


,metsig,metsigres,met,met_noJVT,dmet,ph_pt,ph_eta,ph_phi,jet_central_eta,jet_central_pt1,...,jetterm_sumet,n_jet_central,dphi_met_central_jet,balance,dphi_jj,BDTScore,mt,weights,process,label
0,7.541454,18024.281250,135929.296875,135929.296875,0.0,111165.765625,0.427730,1.273615,0.472236,255010.281250,...,341357.468750,2,2.583472,0.855935,1.248845,0.318229,108.352806,0.110929,ggHyyd,1
1,8.294333,14323.802734,118806.382812,118806.382812,0.0,128489.460938,1.221359,-1.319481,0.386373,135265.062500,...,253867.125000,4,2.676078,1.057630,0.798897,0.287811,112.286324,0.090161,ggHyyd,1
2,8.560959,19878.953125,170182.906250,170182.906250,0.0,54408.984375,1.359410,0.174783,2.078004,120635.601562,...,261521.312500,4,2.937734,1.114419,0.574410,0.301751,108.929871,0.110889,ggHyyd,1
3,10.974058,15954.531250,175085.953125,175085.953125,0.0,51137.843750,1.013092,1.965874,-1.558384,154163.515625,...,196830.500000,2,2.690091,1.149333,0.831215,0.230347,119.204193,0.114817,ggHyyd,1
4,7.569345,17412.738281,131803.031250,131803.031250,0.0,58650.468750,0.973062,0.697516,2.018535,123392.140625,...,258656.484375,1,2.791143,1.543482,-999.000000,0.207851,120.607635,0.126050,ggHyyd,1


In [45]:
df_all.describe()

,metsig,metsigres,met,met_noJVT,dmet,ph_pt,ph_eta,ph_phi,jet_central_eta,jet_central_pt1,...,jetterm,jetterm_sumet,n_jet_central,dphi_met_central_jet,balance,dphi_jj,BDTScore,mt,weights,label
count,70738.000000,7.073800e+04,7.073800e+04,7.073800e+04,70738.000000,7.073800e+04,70738.000000,70738.000000,70738.000000,7.073800e+04,...,7.073800e+04,7.073800e+04,70738.000000,70738.000000,70738.000000,70738.000000,70738.000000,70738.000000,70738.000000,70738.000000
mean,13.219261,2.078370e+04,2.906812e+05,2.915203e+05,839.112549,1.329637e+05,0.805421,0.008817,-0.843758,3.267015e+05,...,3.679446e+05,4.251738e+05,2.253555,1.916518,0.272594,-286.700364,0.283703,184.601990,0.074573,0.030351
std,4.783896,9.639962e+03,2.518426e+05,2.519778e+05,6105.418945,1.521617e+05,0.486169,1.817745,29.102767,2.476370e+05,...,2.565417e+05,2.861762e+05,1.026104,29.164381,29.118479,452.704445,0.053199,96.845467,0.608568,0.171554
min,7.000263,9.143066e+03,1.000023e+05,8.257015e+04,-19999.796875,5.000342e+04,0.000034,-3.141541,-999.000000,-9.990000e+02,...,0.000000e+00,0.000000e+00,0.000000,-999.000000,-999.000000,-999.000000,0.100061,95.001152,-6.165746,0.000000
25%,9.686352,1.518401e+04,1.596632e+05,1.602842e+05,0.000000,6.750854e+04,0.393643,-1.561758,-0.776582,1.757419e+05,...,2.085400e+05,2.415482e+05,1.000000,2.627509,0.973760,-999.000000,0.253362,121.271830,0.006936,0.000000
50%,12.173076,1.867912e+04,2.278787e+05,2.286844e+05,0.000000,9.298250e+04,0.782120,0.010966,0.002814,2.596548e+05,...,2.976765e+05,3.501475e+05,2.000000,2.807345,1.081166,0.499538,0.292076,155.714287,0.027133,0.000000
75%,15.688090,2.405491e+04,3.473008e+05,3.482907e+05,0.000000,1.440316e+05,1.177377,1.576447,0.781779,3.993274e+05,...,4.442719e+05,5.196209e+05,3.000000,2.960182,1.213040,1.253715,0.322001,214.375942,0.082715,0.000000
max,189.987015,1.394890e+06,2.687834e+07,2.687834e+07,49947.515625,3.875383e+06,1.749990,3.141564,2.499738,4.422654e+06,...,4.394674e+06,5.107923e+06,4.000000,3.141564,34.062445,2.499977,0.437586,1781.737549,111.573891,1.000000


In [46]:
df_all.to_csv('../BDT_input.csv', index=False)

In [ ]:
def sel(tot):
    tot2 = []
    for i in range(len(tot)):
        fb2 = tot[i]
        
        metsig_tmp = fb2['met_tst_sig'] # added cut 2 
        mask1 = metsig_tmp >= 7
        fb2 = fb2[mask1]

        dphi_met_phterm_tmp = np.arccos(np.cos(fb2['met_tst_phi'] - fb2['met_phterm_phi'])) # added cut 3
        fb2 = fb2[dphi_met_phterm_tmp >= 1.35]

        dmet_tmp = fb2['met_tst_noJVT_et'] - fb2['met_tst_et'] # added cut 4
        mask1 = dmet_tmp >= -20000
        mask2 = dmet_tmp <= 50000
        fb2 = fb2[mask1 * mask2]

        dphi_met_jetterm_tmp = np.where(fb2['met_jetterm_et'] != 0,   # added cut 5
                                np.arccos(np.cos(fb2['met_tst_phi'] - fb2['met_jetterm_phi'])),
                                -999)
        fb2 = fb2[dphi_met_jetterm_tmp <= 0.70]

        ph_eta_tmp = np.abs(ak.firsts(fb2['ph_eta'])) # added cut 6
        fb2 = fb2[ph_eta_tmp <= 1.75]

        phi1_tmp = ak.firsts(fb2['jet_central_phi']) # added cut 7
        phi2_tmp = ak.mask(fb2['jet_central_phi'], ak.num(fb2['jet_central_phi']) >= 2)[:, 1] 
        dphi_tmp = np.arccos(np.cos(phi1_tmp - phi2_tmp))
        dphi_jj_tmp = ak.fill_none(dphi_tmp, -999)
        fb2 = fb2[dphi_jj_tmp <= 2.5]

        jet_sum_tmp = ak.sum(fb2['jet_central_pt'], axis=-1)
        expr = (fb2['met_tst_et'] + ak.firsts(fb2['ph_pt'])) / ak.where(jet_sum_tmp != 0, jet_sum_tmp, 1)
        balance_tmp = ak.where(jet_sum_tmp != 0, expr, 999) 
        fb2 = fb2[balance_tmp >= 0.65]

        # mt_tmp = np.sqrt(2 * fb2['met_tst_et'] * ak.firsts(fb2['ph_pt']) * 
        #                     (1 - np.cos(fb2['met_tst_phi'] - ak.firsts(fb2['ph_phi'])))) / 1000
        # mask1 = mt_tmp >= 95
        # fb2 = fb2[mask1]

        
        tot2.append(fb2)
    return tot2

tot2 = sel(tot)

signal_name = 'ggHyyd'  # Define signal dataset
cut_name = 'n-1'

def getCutDict():
    cut_dict = {}

    # cut_dict['BDTScore'] = {
    #     'lowercut': np.arange(0, 0.4+0.1, 0.1) # BDTScore > cut
    # }
    # cut_dict['balance'] = {
    #     'lowercut': np.arange(0, 1.5 + 0.05, 0.05), # balance > cut
    #     'uppercut': np.arange(5, 8 + 0.2, 0.2) # balance < cut
    # }
    # cut_dict['dmet'] = {
    #     'lowercut': np.arange(-30000, 0 + 5000, 5000), # dmet > cut
    #     'uppercut': np.arange(10000, 100000 + 5000, 5000), # -10000 < dmet < cut
    # }
    # cut_dict['dphi_jj'] = {
    #     'uppercut': np.arange(1, 3.1 + 0.1, 0.1) # dphi_jj < cut
    # }
    # cut_dict['dphi_met_jetterm'] = {
    #     'lowercut': np.arange(0, 1 + 0.05, 0.05), # dphi_met_jetterm > cut 
    #     'uppercut': np.arange(0.5, 2 + 0.05, 0.05), # dphi_met_jetterm < cut 
    # }
    # cut_dict['dphi_met_phterm'] = {
    #     'lowercut': np.arange(1, 2 + 0.05, 0.05), # dphi_met_phterm > cut
    #     'uppercut': np.arange(2, 3.1 + 0.1, 0.1), # dphi_met_phterm < cut
    # }
    # cut_dict['dphi_ph_centraljet1'] = {
    #     'lowercut': np.arange(0, 2.5 + 0.1, 0.1), # dphi_ph_centraljet1 > cut
    #     'uppercut': np.arange(1.5, 3.1 + 0.1, 0.1) # dphi_ph_centraljet1 < cut
    # }
    # cut_dict['dphi_phterm_jetterm'] = {
    #     'lowercut': np.arange(1, 2.5 + 0.05, 0.05), # dphi_phterm_jetterm > cut
    #     'uppercut': np.arange(2, 4 + 0.1, 0.1) # dphi_phterm_jetterm < cut
    # }
    # cut_dict['met'] = {
    #     'lowercut': np.arange(100000, 140000 + 5000, 5000),  # met > cut
    #     'uppercut': np.arange(140000, 300000 + 5000, 5000),  # met < cut
    # }
    # cut_dict['metsig'] = {
    #     'lowercut': np.arange(0, 10 + 1, 1), # metsig > cut
    #     'uppercut': np.arange(10, 30 + 1, 1), # metsig < cut 
    # }
    cut_dict['mt'] = {
        'lowercut': np.arange(80, 130+5, 5), # mt > cut
        'uppercut': np.arange(120, 230+5, 5) # mt < cut
    }
    # cut_dict['n_jet_central'] = {
    #     'uppercut': np.arange(0, 8+1, 1) # njet < cut
    # }
    # cut_dict['ph_eta'] = {
    #     'uppercut': np.arange(1, 2.5 + 0.05, 0.05), # ph_eta < cut
    # }
    # cut_dict['ph_pt'] = {
    #     'lowercut': np.arange(50000, 100000 + 5000, 5000),  # ph_pt > cut
    #     'uppercut': np.arange(100000, 300000 + 10000, 10000),  # ph_pt > cut
    # }

    return cut_dict
cut_config = getCutDict()

def calculate_significance(cut_var, cut_type, cut_values):
    significance_values = []
    significance_acceptance_values = []
    acceptance_values = []  # Store acceptance percentages

    for cut in cut_values:
        sig_after_cut = 0
        bkg_after_cut = []
        sig_events = 0
        
        for i in range(len(ntuple_name)-1): # not include dijet
            fb = tot2[i]
            process = ntuple_name[i]
            var_config = getVarDict(fb, process, var_name=cut_var)
            x = var_config[cut_var]['var']
            mask = x != -999 # Apply cut: Remove -999 values 
            x = x[mask]

            if process == signal_name:
                sig_events = getWeight(fb, process)
                sig_events = sig_events[mask]
                if cut_type == 'lowercut':
                    mask = x >= cut
                elif cut_type == 'uppercut':
                    mask = x <= cut
                else:
                    raise ValueError("Invalid cut type")
                sig_after_cut = ak.sum(sig_events[mask])
            
            else:
                bkg_events = getWeight(fb, process)
                bkg_events = bkg_events[mask]
                if cut_type == 'lowercut':
                    mask = x >= cut
                elif cut_type == 'uppercut':
                    mask = x <= cut
                else:
                    raise ValueError("Invalid cut type")
                bkg_after_cut.append(ak.sum(bkg_events[mask]))

        # Calculate significance
        significance = sig_after_cut / np.sqrt(sum(bkg_after_cut)) if sum(bkg_after_cut) > 0 else 0

        # Acceptance: ratio of surviving signal events
        acceptance = sig_after_cut / sum(sig_events) if sum(sig_events) > 0 else 0
        acceptance_values.append(acceptance * 100)  # Convert to percentage

        significance_values.append(significance)
        significance_acceptance_values.append(significance * acceptance)

    return significance_values, significance_acceptance_values, acceptance_values

# Compute significance for each variable dynamically
for cut_var, cut_types in cut_config.items():
    for cut_type, cut_values in cut_types.items():
        significance_values, significance_acceptance_values, acceptance_values = calculate_significance(cut_var, cut_type, cut_values)

        # # Reverse order for uppercut plots
        # if cut_type == 'uppercut':
        #     cut_values = cut_values[::-1]
        #     significance_values = significance_values[::-1]
        #     significance_acceptance_values = significance_acceptance_values[::-1]
        #     acceptance_values = acceptance_values[::-1]

        # Plot results
        fig, (ax_top, ax_bot) = plt.subplots(2, 1, figsize=(8, 10), sharex=True)

        # Top plot: Significance vs. Cut
        ax_top.plot(cut_values, significance_values, marker='o', label='Significance')
        ax_top.set_ylabel('Significance')
        ax_top.set_title(f'Significance vs. {cut_var} ({cut_type})')
        ax_top.legend()
        ax_top.grid(True)

        # Bottom plot: Significance * Acceptance vs. Cut
        ax_bot.plot(cut_values, significance_acceptance_values, marker='s', color='r', label='Significance × Acceptance')
        for i, txt in enumerate(acceptance_values):
            ax_bot.text(cut_values[i], significance_acceptance_values[i], f'{txt:.1f}%', 
                        fontsize=10, ha='right', va='bottom', color='purple')
            
        ax_bot.set_xlabel(f'{cut_var} Cut')
        ax_bot.set_ylabel('Significance × Acceptance')
        ax_bot.set_title(f'Significance × Acceptance vs. {cut_var} ({cut_type})')
        
        ax_bot.set_xticks(cut_values)
        ax_bot.set_xticklabels(ax_bot.get_xticks(), rotation=45, ha='right')
        ax_bot.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        # ax_bot.xaxis.set_major_locator(ticker.MaxNLocator(nbins=15))  # Show at most 10 x-ticks
        
        var_configs_tmp = getVarDict(tot2[0], signal_name, cut_var)
        ax_bot.set_xlabel(var_configs_tmp[cut_var]['title'])
        ax_bot.legend()
        ax_bot.grid(True)

        plt.tight_layout()
        plt.savefig(f"../lumi135/mc23d_{cut_name}cut/significance_{cut_var}_{cut_type}.png")
        print(f"Successfully saved to lumi135/mc23d_{cut_name}cut/significance_{cut_var}_{cut_type}.png")
        plt.close()

Successfully saved to lumi135/mc23d_n-1cut/significance_mt_lowercut.png
Successfully saved to lumi135/mc23d_n-1cut/significance_mt_uppercut.png


## Back-up

In [145]:
jet_sum_tmp = ak.sum(fb['jet_central_pt'], axis=-1)
mask = jet_sum_tmp != 0 # True for events where jet_sum is not 0.
balance = (fb['met_tst_et'][mask] + ak.firsts(fb['ph_pt'])[mask]) / jet_sum_tmp[mask]

In [146]:
def findnone(arr):
    mask = ak.is_none(arr)

    n_none = ak.sum(mask)
    print("Number of None values:", n_none)

In [201]:
# Compute sqrt(B_counts) in a safe way
sqrt_B = np.sqrt(B_counts)
# Use np.where to avoid division by zero:
sig_per_bin = np.where(B_counts > 0, S_counts / sqrt_B, 0)
# Replace any possible NaNs with 0:
sig_per_bin = np.nan_to_num(sig_per_bin)



/home/jlai/.local/lib/python3.9/site-packages/awkward/_nplikes/array_module.py:251: RuntimeWarning: invalid value encountered in divide
  return impl(*broadcasted_args, **(kwargs or {}))


In [ ]:
var = {
    'ntuple': [ntuple_name[i]] * len(fb),
    'vtx_sumPt': fb['vtx_sumPt'],
    'n_ph': fb['n_ph'],
    'n_ph_baseline': fb['n_ph_baseline'],
    'n_el_baseline': fb['n_el_baseline'],
    'n_mu_baseline': fb['n_mu_baseline'],
    'n_tau_baseline': fb['n_tau_baseline'],
    'puWeight': fb['pu_weight'],
    'actualIntPerXing': fb['actualIntPerXing'],
    'mt': np.sqrt(2 * fb['met_tst_et'] * ak.firsts(fb['ph_pt']) * (1 - np.cos(fb['met_tst_phi'] - ak.first(fb['ph_phi'])))) / 1000,
    'metsig': fb['met_tst_sig'],
    'metsigres': fb['met_tst_et'] / fb['met_tst_sig'],
    'met': fb['met_tst_et'] + 50000,  # applying the '+50000' shift???
    'met_noJVT': fb['met_tst_noJVT_et'],
    'met_cst': fb['met_cst_et'],
    'met_track': fb['met_track_et'],
    'dmet': fb['met_tst_noJVT_et'] - fb['met_tst_et'],
    'ph_pt': ak.first(fb['ph_pt']) - 150000,  # applying the '-150000' shift???
    'ph_eta': np.abs(ak.first(fb['ph_eta'])),
    'ph_phi': ak.first(fb['ph_phi']),
    'jet_central_eta': ak.first(fb['jet_central_eta']),
    'jet_central_pt1': ak.first(fb['jet_central_pt']),
    'jet_central_pt2': fb['jet_central_pt'][:, 1],
    'jet_central_pt': fb['jet_central_pt'],
    'dphi_met_phterm': np.arccos(np.cos(fb['met_tst_phi'] - fb['met_phterm_phi'])),
    'dphi_met_ph': np.arccos(np.cos(fb['met_tst_phi'] - ak.first(fb['ph_phi']))),
    'dphi_met_jetterm': np.where(fb['met_jetterm_et'] != 0,
                                  np.arccos(np.cos(fb['met_tst_phi'] - fb['met_jetterm_phi'])),
                                  0),
    'dphi_phterm_jetterm': np.where(fb['met_jetterm_et'] > 0,
                                    np.arccos(np.cos(fb['met_phterm_phi'] - fb['met_jetterm_phi'])),
                                    4),
    'dphi_ph_centraljet1': np.arccos(np.cos(ak.first(fb['ph_phi']) - ak.first(fb['jet_central_phi']))),
    'dphi_ph_jet1': np.arccos(np.cos(ak.first(fb['ph_phi']) - ak.first(fb['jet_central_phi']))),
    'dphi_central_jet1_jet2': np.arccos(np.cos(fb['jet_central_phi'][0] - fb['jet_central_phi'][1])),
    'metplusph': fb['met_tst_et'] + ak.first(fb['ph_pt']),
    'failJVT_jet_pt': fb['failJVT_jet_pt'],
    'failJVT_jet_pt1': ak.first(fb['failJVT_jet_pt']),
    'softerm': fb['met_softerm_tst_et'],
    'jetterm': fb['met_jetterm_et'],
    'jetterm_sumet': fb['met_jetterm_sumet'],
    'n_jet': fb['n_jet'],
    'n_jet_central': fb['n_jet_central'],
    'n_jet_fwd': fb['n_jet'] - fb['n_jet_central'],
    'vertex': np.abs(fb['pv_truth_z'][0] - fb['pv_z'][0]) == np.min(np.abs(fb['pv_truth_z'][0] - fb['pv_z'])),
    'goodPV': np.abs(fb['pv_truth_z'][0] - fb['pv_z'][0]) <= 0.5,
    'dphi_met_central_jet': np.arccos(np.cos(fb['met_tst_phi'] - ak.first(fb['jet_central_phi']))),
    'counts': 0.5,
    'jet_central_timing1': ak.first(fb['jet_central_timing']),
    'jet_central_timing': fb['jet_central_timing'],
    'jet_central_emfrac': fb['jet_central_emfrac'],
    'jet_central_emfrac1': ak.first(fb['jet_central_emfrac']),
    'balance': (fb['met_tst_et'] + ak.first(fb['ph_pt'])) / np.sum(fb['jet_central_pt']),
    'balance_sumet': (fb['met_tst_et'] + ak.first(fb['ph_pt'])) / fb['met_jetterm_sumet'],
    'central_jets_fraction': np.where(fb['n_jet'] > 0, fb['n_jet_central'] / fb['n_jet'], -1),
    'trigger': fb['trigger_HLT_g50_tight_xe40_cell_xe70_pfopufit_80mTAC_L1eEM26M'],
    'dphi_jj': np.arccos(np.cos(fb['jet_central_phi'][1] - fb['jet_central_phi'][0])) if len(fb['jet_central_phi']) > 1 else -1,
}


In [ ]:
dict={}
dict['allcut'] = {
    'str': (
        'met_tst_et > 100000 && '
        'trigger_HLT_g50_tight_xe40_cell_xe70_pfopufit_80mTAC_L1eEM26M && '
        'ph_pt[0] > 50000 && ' 
        'ph_pt[0] < 200000 && '
        '(met_tst_noJVT_et-met_tst_et) > -60000 && ' # dmet
        '(met_tst_noJVT_et-met_tst_et) < 60000 && '
        'sqrt(2*met_tst_et*ph_pt[0]*(1-cos(met_tst_phi-ph_phi[0])))/1000 > 40 && ' # mt
        'sqrt(2*met_tst_et*ph_pt[0]*(1-cos(met_tst_phi-ph_phi[0])))/1000 < 200 && '
        '((met_jetterm_et!=0)*Alt$(acos(cos(met_tst_phi-met_jetterm_phi)),0)+(met_jetterm_et==0)*0) < 1.5 && ' # dphi_met_jetterm
        '(met_tst_et+ph_pt[0])/Sum$(jet_central_pt) <= 12' # balance
    )}
dict['allcut']['str']